## Word2Vec LAC 2017 (concise case)

modelling concise case of LAC 2017 use FastText 

In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from gensim.models import Word2Vec
import ast
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.externals import joblib

C:\Users\Aska\Anaconda3\envs\py36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import model_from_json
from keras.callbacks import CSVLogger

from gensim.models.wrappers import FastText

Using TensorFlow backend.


In [3]:
DATA_PATH="../Data/Raw/"
DATA_PATH2="../..//Data/Manipulation/"
MODEL_PATH="../../Models/"
EMB_PATH="../../Vocab_FastText/"

### Word2Vec Vectorize

In [4]:
dta_tr=pd.read_csv(DATA_PATH2+"data_train_tokenize_concise_only2_train.csv")
dta_te=pd.read_csv(DATA_PATH2+"data_train_tokenize_concise_only2_test.csv")

In [5]:
dta_tr["true_text"]=dta_tr.token_short_des.apply(lambda x: " ".join(ast.literal_eval(x)))
dta_te["true_text"]=dta_te.token_short_des.apply(lambda x: " ".join(ast.literal_eval(x)))

In [8]:
# embd_f= Word2Vec.load(EMB_PATH+"cc.en.300.bin")
embd_f=FastText.load_fasttext_format(EMB_PATH+"cc.en.300.bin")

UnpicklingError: invalid load key, '\xba'.

In [ ]:
# s0=dta_tr.token_short_des.tolist()
# sentence=[ast.literal_eval(x) for x in s0]

In [ ]:
# %%time
# model_emb75 = Word2Vec(sentence,size=75,
#         window=8,min_count=10,workers=5)

# model_emb75.train(sentence, total_examples=len(sentence), epochs=8)

In [ ]:
# %%time
# model_emb150 = Word2Vec(sentence,size=150,
#         window=8,min_count=10,workers=5)

# model_emb150.train(sentence, total_examples=len(sentence), epochs=8)

In [ ]:
# %%time
# model_emb225 = Word2Vec(sentence,size=225,
#         window=8,min_count=10,workers=5)

# model_emb225.train(sentence, total_examples=len(sentence), epochs=8)

In [ ]:
# model_emb4.wv["apple"]

In [ ]:
# model_emb75.save(MODEL_PATH+'concise_vocab_75.bin')
# model_emb150.save(MODEL_PATH+'concise_vocab_150.bin')
# model_emb225.save(MODEL_PATH+'concise_vocab_225.bin')

In [ ]:
# model_emb150.layer1_size

### Modeling Part

In [ ]:
EPOCH=8
BATCH=595

In [ ]:
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, embedding.layer1_size))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        try:
            weight_matrix[i] = embedding[word]
        except:
            weight_matrix[i] = np.zeros(embedding.layer1_size)*0.0001
    return weight_matrix

def Emb_Lay(X_train,tokenizer,model_emb):
#     max_length = 20
    max_length = 25
    encoded_docs = tokenizer.texts_to_sequences(X_train)
    Xtr2 = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

    vocab_size = len(tokenizer.word_index) + 1
    embedding_vectors = get_weight_matrix(model_emb, tokenizer.word_index)
    embedding_layer = Embedding(vocab_size, model_emb.layer1_size, weights=[embedding_vectors], 
                                input_length=max_length, trainable=False)
    return embedding_layer,Xtr2

def cur_model(X,tokenizer,emb):
    Emb_Layer=Emb_Lay(X,tokenizer,emb)
    model = Sequential()
    model.add(Emb_Layer[0])
#     model.add(Conv1D(filters=24, kernel_size=6, activation='relu'))
    model.add(Conv1D(filters=380, kernel_size=10, activation='relu'))
#     model.add(Conv1D(filters=100, kernel_size=5, activation='relu'))

#     model.add(MaxPooling1D(pool_size=3))
    model.add(Flatten())
    model.add(Dropout(0.7))
    
    model.add(Dense(200, activation='relu'))
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(70, activation='relu'))
    model.add(Dense(70, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    print(model.summary())
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model,Emb_Layer[1]

In [ ]:
Y_true=dta_tr["conci"]
X_tr=dta_tr.true_text

In [ ]:
## Tokenizer

tokenizer_x = Tokenizer()
tokenizer_x.fit_on_texts(X_tr)

In [ ]:
## Model 75
model75,X75=cur_model(X_tr,tokenizer_x,model_emb75)

In [ ]:
%%time
csv_logger = CSVLogger(DATA_PATH2+'wemb_log75.csv', append=True, separator=',')
model75.fit(X75, Y_true, epochs=EPOCH, batch_size=BATCH,verbose=0,callbacks=[csv_logger])

In [ ]:
## Model 150
model150,X150=cur_model(X_tr,tokenizer_x,model_emb150)

In [ ]:
%%time
csv_logger = CSVLogger(DATA_PATH2+'wemb_log150.csv', append=True, separator=',')
model150.fit(X150, Y_true, epochs=EPOCH, batch_size=BATCH,verbose=0,callbacks=[csv_logger])

In [ ]:
## Model 225
model225,X225=cur_model(X_tr,tokenizer_x,model_emb225)

In [ ]:
%%time
csv_logger = CSVLogger(DATA_PATH2+'wemb_log225.csv', append=True, separator=',')
model225.fit(X225, Y_true, epochs=EPOCH, batch_size=BATCH,verbose=0,callbacks=[csv_logger])

In [ ]:
# Model Saving

# serialize model to JSON
for model,mod_name in [(model75,"wemb_model75"),(model150,"wemb_model150"),(model225,"wemb_model225")]:
    model_json = model.to_json()
    with open(MODEL_PATH+mod_name+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(MODEL_PATH+mod_name+"_weight.h5")
    print("Saved model to disk")

### Evaluate

In [ ]:
for model,X in [(model75,X75),(model150,X150),(model225,X225)]:
    loss, acc = model.evaluate(X, Y_true, verbose=0)
    print('Train Accuracy: %f' % (acc*100))

In [ ]:
X_te=dta_te.true_text

X75_te=Emb_Lay(X_te,tokenizer_x,model_emb75)[1]
X150_te=Emb_Lay(X_te,tokenizer_x,model_emb150)[1]
X225_te=Emb_Lay(X_te,tokenizer_x,model_emb225)[1]

Y_test=dta_te.conci 

In [ ]:
for model,X in [(model75,X75_te),(model150,X150_te),(model225,X225_te)]:
    loss, acc = model.evaluate(X, Y_test, verbose=0)
    print('Test Accuracy: %f' % (acc*100))

In [ ]:
# https://fasttext.cc/docs/en/crawl-vectors.html